In [ ]:
#Open API Fine Tuning Dashboard URL: https://platform.openai.com/finetune/


In [2]:
from dotenv import load_dotenv
from openai import OpenAI
import os

# --- Load API Key ---
load_dotenv(override=True, dotenv_path="../.env")
my_api_key = os.getenv("OPENAI_API_KEY")


client = OpenAI(api_key=my_api_key)

In [3]:
with open("/Users/kannans100/sansa/Session_07/data/training_data.jsonl", "rb") as f:
    training_file_obj = client.files.create(file=f, purpose="fine-tune")

print("Uploaded training file ID:", training_file_obj.id)

with open("/Users/kannans100/sansa/Session_07/data/validation_data.jsonl", "rb") as f:
    validation_file_obj = client.files.create(file=f, purpose="fine-tune")

print("Uploaded validation file ID:", validation_file_obj.id)

Uploaded training file ID: file-WSVQqw78XfQqkGJ3VdHF24
Uploaded validation file ID: file-9sm211K7G3No4LVvnZStoo


In [4]:
# Create a fine-tuning job
job = client.fine_tuning.jobs.create(
    training_file=training_file_obj.id,       # The file ID you uploaded
    validation_file=validation_file_obj.id,     # Optional
    model="gpt-4.1-nano-2025-04-14",               # or "gpt-3.5-turbo", etc.
    suffix="brand-customer-support"       # Optional model name suffix
)

print("Fine-tune job created:", job.id)

Fine-tune job created: ftjob-9EnUymExH97BCUeiFd6XCFBn


In [5]:
jobs = client.fine_tuning.jobs.list(limit=5)
for j in jobs.data:
    print(j.id, j.status, j.fine_tuned_model)

ftjob-9EnUymExH97BCUeiFd6XCFBn validating_files None


In [8]:
def ask_question_without_finetuning(prompt):
    print(f"User asked: {prompt}")
    response = client.chat.completions.create(
        model="gpt-5-nano",
        messages=[
            {"role": "system", "content": "You are a helpful assistant. Answer as concisely as possible."},
            {"role": "user", "content": prompt}
        ]
    )
    print (response)
    return response.choices[0].message.content  

def ask_question_with_finetuning(prompt):
    print(f"User asked: {prompt}")
    response = client.chat.completions.create(
        model="ft:gpt-4.1-nano-2025-04-14:home:brand-customer-support:CjqxRuPk",
        messages=[
            {"role": "system", "content": "You are a helpful assistant. Answer as concisely as possible."},
            {"role": "user", "content": prompt}
        ]
    )
    print (response)
    return response.choices[0].message.content  

In [ ]:
import time
while True:
    # Ask user for a question
    user_prompt = input("Ask something: ")

    if (user_prompt.lower() != 'quit'):
        # Get and print the response
        response = ask_question_without_finetuning(user_prompt)
        print("\n[WITHOUT FINE TUNING] OpenAI says:", response)

        response = ask_question_with_finetuning(user_prompt)
        print("\n[WITH FINE TUNING] OpenAI says:", response)

        # add delay of 3 seconds
        time.sleep(3)
    else:
        break    

User asked: what happened to my order
ChatCompletion(id='chatcmpl-CjrBgZsZJU3v0U9RwS21WeKeZyNPm', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='I don’t have access to your order, but I can help. Please provide:\n- Your order number (or the email/name on the order)\n- The retailer or platform\n\nIf you’d rather, you can check quickly by logging in: Account > Orders > the specific order.\n\nCommon statuses and what they mean:\n- Processing/Pending: order is being prepared\n- Packed/Shipped: on the way; you should see a tracking number\n- Out for delivery: with the carrier today\n- Delivered: arrived at the address\n- Canceled/Backordered/On hold: there’s an issue needing attention\n\nIf status is unclear or stuck, I can guide you on what to check or draft a message to customer support.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1765045248, model='gpt-5-nano-2025-08-07